# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">CONVOLVE 3.0 Round 2 </h1></span>

# Importing Important Libraries

In [ ]:
import pandas as pd
import numpy as np
#=====================#
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import warnings
warnings.filterwarnings("ignore")


# EXPLORATORY DATA ANALYSIS AND CREATING TARGET COLUMN USING train_action_history

In [ ]:
train_history=pd.read_csv("train_action_history.csv")
print(train_history.shape)
train_history.head()

In [ ]:
train_history.columns

In [ ]:
train_cdna=pd.read_csv("train_cdna_data.csv")
print(train_cdna.shape)
train_cdna.head()

In [ ]:
for x in train_cdna.columns:
    print(x)

In [ ]:
train_history['customer_code'].nunique()

In [ ]:
train_cdna['CUSTOMER_CODE'].nunique()

In [ ]:
train_cdna['batch_date'].unique()

In [ ]:
print(train_history['send_timestamp'].min())
print(train_history['send_timestamp'].max())

In [ ]:
customer_data=train_cdna['CUSTOMER_CODE'].value_counts()
for customer,value in customer_data.items():
    print(customer,value)

### Removing customers which donot have data in train_cdna

In [ ]:
filtered_train_history = train_history[train_history['customer_code'].isin(train_cdna['CUSTOMER_CODE'])]

In [ ]:
filtered_train_history['customer_code'].nunique()

In [ ]:
filtered_train_history.shape

In [ ]:
print(set(filtered_train_history['customer_code']) - set(train_cdna['CUSTOMER_CODE']))

### Mapping the Slots to values between 1 to 28 with 0 indicating not opened

In [ ]:
filtered_train_history.loc[:, 'send_timestamp'] = pd.to_datetime(filtered_train_history['send_timestamp'])
filtered_train_history.loc[:, 'open_timestamp'] = pd.to_datetime(filtered_train_history['open_timestamp'])

In [ ]:
filtered_trained_history_time = filtered_train_history[(filtered_train_history['open_timestamp'].isna()) | (filtered_train_history['open_timestamp'] > filtered_train_history['send_timestamp'])]

In [ ]:
filtered_train_history.head()

In [ ]:
# Define function to map timestamp to time slot
def map_to_slot(timestamp):
    if pd.isna(timestamp):
        return 0  # NaT gets slot 0
    # Extract the day of the week (0=Monday, 6=Sunday) and hour
    day_of_week = timestamp.weekday()  # Monday = 0, Sunday = 6
    hour = timestamp.hour

    # Determine the time slot based on the hour
    if 9 <= hour < 12:
        slot_in_day = 1
    elif 12 <= hour < 15:
        slot_in_day = 2
    elif 15 <= hour < 18:
        slot_in_day = 3
    elif 18 <= hour < 21:
        slot_in_day = 4
    else:
        return 0  # Time outside defined slots

    # Calculate the overall slot number
    return day_of_week * 4 + slot_in_day

filtered_train_history = filtered_train_history.copy()
filtered_train_history['time_slot'] = filtered_train_history['open_timestamp'].apply(map_to_slot)


In [ ]:
filtered_train_history['sent_time_slot']=filtered_train_history['send_timestamp'].apply(map_to_slot)
filtered_train_history

### Creating a batch_date_history column so that it can be joined with the corresponding batch_dates of the train_cdna

In [ ]:
# Convert send_timestamp to datetime
filtered_train_history['date_send'] = pd.to_datetime(filtered_train_history['send_timestamp'])

# Vectorized assignment of batch_date_history
conditions = [
    filtered_train_history['date_send'] < '2024-06-28',
    filtered_train_history['date_send'] < '2024-07-26',
    filtered_train_history['date_send'] < '2024-08-30',
    filtered_train_history['date_send'] < '2024-09-27',
    filtered_train_history['date_send'] < '2024-10-25'
]
choices = [
    pd.Timestamp('2024-05-31').date(),
    pd.Timestamp('2024-06-28').date(),
    pd.Timestamp('2024-07-26').date(),
    pd.Timestamp('2024-08-30').date(),
    pd.Timestamp('2024-09-27').date()
]

filtered_train_history['batch_date_history'] = np.select(
    conditions,
    choices,
    default=pd.Timestamp('2024-10-25').date()  # Default value for remaining rows
)


### Creation of Target Column

In [ ]:
# Filter rows where time_slot > 0
filtered_train_history_notzero = filtered_train_history[filtered_train_history['time_slot'] > 0]

# Group and aggregate to get counts for each time_slot
grouped = (
    filtered_train_history_notzero.groupby(['batch_date_history', 'customer_code', 'time_slot'])
    .size()
    .reset_index(name='count')
)

# For each batch_date_history and customer_code, handle ties by taking the median time_slot
def resolve_ties(group):
    max_count = group['count'].max()                 # Find the maximum count
    max_slots = group[group['count'] == max_count]  # Filter rows with the maximum count
    median_slot = max_slots['time_slot'].median()   # Compute the median of the time_slot
    return pd.Series({
        'target_slot': median_slot
    })

# Apply the function to resolve ties and create the target column
target_dataset = grouped.groupby(['batch_date_history', 'customer_code']).apply(resolve_ties).reset_index()

In [ ]:
filtered_row2 = target_dataset[target_dataset['customer_code'] == '0000b844339117db33d8dcc6ca4e97a8c989a918c8cf0183e1b4b6f1e149a2d9']
filtered_row2

In [ ]:
filtered_row = grouped[grouped['customer_code'] == '0000b844339117db33d8dcc6ca4e97a8c989a918c8cf0183e1b4b6f1e149a2d9']
filtered_row

In [ ]:
target_dataset.to_csv("target_median.csv", index=False)

# EXPLORATORY DATA ANALYSIS AND PREPROCESSING OF train_cdna_data

In [ ]:
file_path = "train_cdna_data.csv"
cdna_df = pd.read_csv(file_path)

In [ ]:
# Display initial overview
print("Initial Dataset Overview:")
print(cdna_df.head())
print("\nDataset Info:")
print(cdna_df.info())

Initial Dataset Overview:
                                       CUSTOMER_CODE     v2  v3  v4   v5  \
0  ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...  50-54 NaN NaN   99   
1  6e8e3227297409f3f33578400302825263cadc2ed0d1a0...  35-39 NaN NaN   ZZ   
2  1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...  20-24 NaN NaN   99   
3  06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...  20-24 NaN NaN  NaN   
4  0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...  30-34 NaN NaN  NaN   

            v6     v7   v8     v9  v10  ... v294  v295 v296 v297 v298 v299  \
0        THANE  99999  NaN  INDIA   IN  ...  187    44  204  150   73  271   
1         Pune     ZZ  NaN  INDIA  411  ...  187    44  204  150   73  271   
2      BHIWANI  95013  NaN  INDIA   IN  ...  187    44  204  150   73  271   
3  CHIKMAGALUR    NaN  NaN  INDIA  NaN  ...  187    44  204  150   73  271   
4        NASIK    NaN  NaN  INDIA  NaN  ...  187    44  204  150   73  271   

  v300 v301 v302  batch_date  
0  164  170  246 

### Handle Age Range(Mean)

Column v2 :  Contains age range. Replaced range with mean.

In [ ]:
# Example: Assume column `v2` contains age ranges like '50-54'
if 'v2' in cdna_df.columns:
    cdna_df['age_mean'] = cdna_df['v2'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2 if isinstance(x, list) else np.nan)
    cdna_df.drop('v2', axis=1, inplace=True)
    print("\nProcessed Age Range into Mean:")
    print(cdna_df[['age_mean']].head())



Processed Age Range into Mean:
   age_mean
0      52.0
1      37.0
2      22.0
3      22.0
4      32.0


### Drop Columns with Significant NaN Values

To address missing values in the dataset, we identified and removed columns between 60-90% missing values and those over 95% missing values.

In [ ]:
null_count=pd.DataFrame(cdna_df.isnull().sum())
null_count.columns=['count']
null_count

,count
CUSTOMER_CODE,0
v3,1285402
v4,1285402
v5,441673
v6,3907
...,...
v300,0
v301,0
v302,0
batch_date,0


In [ ]:
for index, row in null_count.iterrows():
    for col_name in null_count.columns:
        if (row[col_name]!=0):
            print(index,row[col_name])

v3 1285402
v4 1285402
v5 441673
v6 3907
v7 392784
v8 1285391
v9 21028
v10 181687
v11 12
v12 1285390
v13 1285385
v14 1285385
v15 688538
v16 1285151
v17 1284237
v18 1285057
v19 1284857
v20 1284934
v21 1285402
v22 1285402
v23 1285402
v24 404144
v25 1285402
v26 1285402
v27 60259
v28 1285385
v29 234294
v30 66857
v31 50456
v32 541819
v33 210781
v34 195766
v35 262859
v37 5885
v38 1280876
v42 587777
v43 61223
v44 615507
v46 1285276
v47 1285330
v48 1285276
v54 45203
v55 766354
v56 21152
v57 1285372
v58 1064980
v59 1284736
v60 411490
v61 1084480
v62 1279864
v63 749668
v64 1085902
v65 749668
v66 753484
v67 1269454
v68 753484
v69 753484
v70 1285396
v71 753484
v72 1284994
v73 121870
v74 753484
v75 1285396
v76 1285402
v77 749668
v78 1285402
v79 760659
v80 45240
v81 150
v84 760659
v85 1113196
v99 650317
v101 34374
v102 237706
v103 742352
v107 1169030
v108 1169030
v109 1169030
v110 941227
v111 941227
v112 941227
v113 1126304
v114 1126304
v115 1126304
v116 880427
v117 880427
v118 880427
v119 1095923
v1

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = cdna_df.isnull().mean() * 100

# Identify columns with missing values between 60% and 90%
cols_to_drop = missing_percentage[(missing_percentage >= 60) & (missing_percentage <= 90)].index

print(f"\nColumns with 60% to 90% missing values: {cols_to_drop.tolist()}")

# Save columns with 60% to 90% missing values in a separate dataset
cdna_df_dropped = cdna_df[cols_to_drop]

# Drop those columns from the original dataset
cdna_df_cleaned = cdna_df.drop(columns=cols_to_drop)

print("\nShape of Dataset with Columns to Drop:")
print(cdna_df_dropped.shape)

print("\nShape of Cleaned Dataset After Dropping Columns:")
print(cdna_df_cleaned.shape)



Columns with 60% to 90% missing values: ['v58', 'v61', 'v64', 'v85', 'v110', 'v111', 'v112', 'v113', 'v114', 'v115', 'v116', 'v117', 'v118', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v125', 'v126', 'v127', 'v134', 'v135', 'v136', 'v140', 'v141', 'v142', 'v146', 'v147', 'v148', 'v171', 'v172', 'v173', 'v174', 'v175', 'v176', 'v177', 'v178', 'v210', 'v211', 'v212', 'v213', 'v215', 'v216']

Shape of Dataset with Columns to Drop:
(1285402, 45)

Shape of Cleaned Dataset After Dropping Columns:
(1285402, 258)


#### Columns with More Than 90% Missing Values

Deducting that the missing values may also hold some significance, we encoded the columns having 90-95% missing data replacing NaNs with 0 and any other value with 1.

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = cdna_df_cleaned.isnull().mean() * 100

# Identify columns with missing values between 95% and 100%
cols_to_drop_95_100 = missing_percentage[(missing_percentage >= 95) & (missing_percentage <= 100)].index

# Drop the columns with 95-100% missing values
cdna_df_cleaned_2 = cdna_df_cleaned.drop(columns=cols_to_drop_95_100)

print(f"\nColumns with 95-100% missing values dropped: {cols_to_drop_95_100.tolist()}")



Columns with 95-100% missing values dropped: ['v3', 'v4', 'v8', 'v12', 'v13', 'v14', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v25', 'v26', 'v28', 'v38', 'v46', 'v47', 'v48', 'v57', 'v59', 'v62', 'v67', 'v70', 'v72', 'v75', 'v76', 'v78', 'v149', 'v150', 'v151', 'v152', 'v153', 'v154', 'v155', 'v156', 'v157', 'v158', 'v159', 'v160', 'v163', 'v164', 'v165', 'v166', 'v167', 'v168', 'v169', 'v170', 'v179', 'v180', 'v181', 'v182', 'v187', 'v188', 'v189', 'v190', 'v191', 'v192', 'v193', 'v194', 'v195', 'v196', 'v197', 'v198', 'v199', 'v200', 'v201', 'v202', 'v214', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v242', 'v245', 'v246', 'v247', 'v248', 'v249', 'v250', 'v251']


In [ ]:
# Identify columns with missing values between 90% and 95%
cols_90_95 = missing_percentage[(missing_percentage >= 90) & (missing_percentage < 95)].index

# Replace NaN values with 0 and the rest with 1 for both numerical and categorical columns
cdna_df_cleaned_2[cols_90_95] = cdna_df_cleaned_2[cols_90_95].apply(lambda x: x.fillna(0) if x.dtype in ['float64', 'int64'] else x.fillna('0'))
cdna_df_cleaned_2[cols_90_95] = cdna_df_cleaned_2[cols_90_95].apply(lambda x: x.apply(lambda val: 1 if val != 0 else 0))

print("\nNaN values in columns with 90-95% missing replaced with 0, other values replaced with 1.")



NaN values in columns with 90-95% missing replaced with 0, other values replaced with 1.


In [ ]:
# Find categorical columns in the 90-95% missing value range
categorical_cols_90_95 = cdna_df_cleaned_2[cols_90_95].select_dtypes(include=['object']).columns

for col in categorical_cols_90_95:
    # Create a new numerical column based on the categorical column
    new_col = col + "_num"
    cdna_df_cleaned_2[new_col] = cdna_df_cleaned_2[col].apply(lambda x: 1 if pd.notnull(x) else 0)

    # Drop the original categorical column
    cdna_df_cleaned_2 = cdna_df_cleaned_2.drop(columns=[col])

    print(f"Processed and dropped categorical column: {col}")


In [ ]:
cdna_df_cleaned_2.head()


,CUSTOMER_CODE,v5,v6,v7,v9,v10,v11,v15,v24,v27,...,v295,v296,v297,v298,v299,v300,v301,v302,batch_date,age_mean
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,99,THANE,99999,INDIA,IN,2018-08-25T00:00:00.000Z,NaN,0.0,MARRIED,...,44,204,150,73,271,164,170,246,2024-09-27,52.0
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,ZZ,Pune,ZZ,INDIA,411,2018-08-03T14:12:16.000Z,Resident Individuals,NaN,Married,...,44,204,150,73,271,164,170,246,2024-09-27,37.0
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,99,BHIWANI,95013,INDIA,IN,2021-10-08T00:00:00.000Z,NaN,0.0,MARRIED,...,44,204,150,73,271,164,170,246,2024-09-27,22.0
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,NaN,CHIKMAGALUR,NaN,INDIA,NaN,2024-03-18T00:00:00.000Z,Resident Individuals,NaN,Unmarried,...,44,204,150,73,271,164,170,246,2024-09-27,22.0
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,NaN,NASIK,NaN,INDIA,NaN,2024-02-26T00:00:00.000Z,Resident Individuals,NaN,Unmarried,...,44,204,150,73,271,164,170,246,2024-09-27,32.0


In [ ]:
# Print the remaining columns in the dataset
print("\nRemaining columns after all cleaning steps:")
print(cdna_df_cleaned_2.columns.tolist())

# Print the shape of the new dataset
print(f"\nShape of the cleaned dataset: {cdna_df_cleaned_2.shape}")



Remaining columns after all cleaning steps:
['CUSTOMER_CODE', 'v5', 'v6', 'v7', 'v9', 'v10', 'v11', 'v15', 'v24', 'v27', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v60', 'v63', 'v65', 'v66', 'v68', 'v69', 'v71', 'v73', 'v74', 'v77', 'v79', 'v80', 'v81', 'v82', 'v83', 'v84', 'v86', 'v87', 'v88', 'v89', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v108', 'v109', 'v128', 'v129', 'v130', 'v131', 'v132', 'v133', 'v137', 'v138', 'v139', 'v143', 'v144', 'v145', 'v161', 'v162', 'v183', 'v184', 'v185', 'v186', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v224', 'v225', 'v226', 'v227', 'v228', 'v229', 'v230', 'v231', 'v232', 'v233', 'v234', 'v235', 'v236', 'v237', 'v238', 'v239', 'v240', 'v241', 'v243', 'v244', 'v252', 'v253', 'v254', 'v255', 'v256', 'v257', 'v258', 'v25

**Identifying columns with mixed data types & categorical columns that also had missing values**

In [ ]:
# Step 1: Identify columns with mixed data types (numerical + alphabetic + NaN)
mixed_data_columns = []

# Check each column for mixed data types
for col in cdna_df_cleaned_2.columns:
    if cdna_df_cleaned_2[col].dtype == 'object':  # Check only string columns
        column_as_str = cdna_df_cleaned_2[col].astype(str)

        # Check if the column contains numeric or alphabetic values (excluding NaN)
        if column_as_str.str.isnumeric().any() and column_as_str.str.isalpha().any():
            mixed_data_columns.append(col)

# Step 2: Identify categorical columns with NaN values
# Get columns that have 'object' type and check for NaN values
categorical_cols_with_nan = cdna_df_cleaned_2.select_dtypes(include=['object']).columns
categorical_cols_with_nan = categorical_cols_with_nan[cdna_df_cleaned_2[categorical_cols_with_nan].isna().any()].tolist()

# Print the results
print("Columns with mixed data types (numerical + alphabetic + NaN values):")
print(mixed_data_columns)

print("\nColumns with categorical data types that have NaN values:")
print(categorical_cols_with_nan)


Columns with mixed data types (numerical + alphabetic + NaN values):
['v5', 'v7', 'v9', 'v10', 'v15', 'v30', 'v31', 'v37', 'v63', 'v229', 'v230']

Columns with categorical data types that have NaN values:
['v5', 'v6', 'v7', 'v9', 'v10', 'v11', 'v15', 'v27', 'v29', 'v30', 'v31', 'v33', 'v34', 'v35', 'v37', 'v42', 'v43', 'v54', 'v55', 'v56', 'v60', 'v63', 'v66', 'v68', 'v69', 'v71', 'v73', 'v74', 'v81', 'v99', 'v101', 'v102', 'v103', 'v229', 'v230', 'v271', 'v272', 'v273', 'v274', 'v275', 'v276', 'v277', 'v278', 'v279', 'v280', 'v281', 'v282', 'v283', 'v284', 'v285', 'v286']


In [ ]:
len(mixed_data_columns)


11

In [ ]:
len(categorical_cols_with_nan)

51

**Handling columns with mixed data types**

Column v5: 1279548 entries were 99.0 clearly being the most frequent entry. A few ‘ZZ’ entries were present. Replaced the NaN values and ‘ZZ’ entries with 99.0

In [ ]:
# Step 1: Convert numerical strings to integers where possible
cdna_df_cleaned_2['v5'] = pd.to_numeric(cdna_df_cleaned_2['v5'], errors='coerce')

# Step 2: Replace 'ZZ' and NaN with the median of the column
median_v5 = cdna_df_cleaned_2['v5'].median()
cdna_df_cleaned_2['v5'].fillna(median_v5, inplace=True)

# Step 3: Print unique values and their counts in the specified format
unique_values_counts = cdna_df_cleaned_2['v5'].value_counts()

# Print the results
for value, count in unique_values_counts.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts.sum()}")


99.0       1279548
31.0       2599
45.0       1223
42.0       450
5.0        434
49.0       307
32.0       275
41.0       260
11.0       79
33.0       66
2.0        36
12.0       34
39.0       17
19.0       12
51.0       12
4.0        12
6.0        12
10.0       6
1.0        6
7.0        6
3.0        6
100.0      2
Total      1285402


Column v7: Dropped v7 because it contained some alphanumeric values like 1159A, etc.

In [ ]:
# Count the unique values in column 'v7'
unique_values_counts_v7 = cdna_df_cleaned_2['v7'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v7.items():
    print(f"{value:<10} {count}")

# Print the total
print("\nTotal")
print(f"{'Total':<10} {unique_values_counts_v7.sum()}")


95013      435514
ZZ         187674
99999      150928
95012      63783
95019      32461
01         5409
95011      1758
95003      1398
52319      1122
01101      796
02         714
52101      709
01103      628
60204      558
93002      482
52313      434
95002      420
01201      390
95014      356
51909      350
04         331
85102      294
85101      280
93001      267
28901      258
74101      246
05         192
06         186
03         186
51303      175
52201      164
52312      149
51204      144
60301      140
60201      126
91101      102
15         102
17209      89
18101      88
93109      84
01202      78
25201      78
93003      77
51302      76
20101      72
36101      72
95009      72
93101      69
27101      65
50002      60
41001      54
11         54
41         49
55201      48
22101      48
52311      47
28101      46
01209      42
21003      42
07         42
01203      42
51406      41
27301      39
29202      36
51202      36
17101      36
24219      36
65939   

In [ ]:
# Drop column 'v7' from the dataset
cdna_df_cleaned_2 = cdna_df_cleaned_2.drop(columns=['v7'], errors='ignore')

# Display the updated shape of the dataset
cdna_df_cleaned_2.shape


(1285402, 171)

Columns v9, v10, v15, v30, v31, v37, 229, 230 : Dropped due to presence of too many categories. One hot encoding would not be helpful in this case.

In [ ]:
# Count the unique values in column 'v9'
unique_values_counts_v9 = cdna_df_cleaned_2['v9'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v9.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v9.sum()}")


INDIA      1113178
IN         124878
India      10996
UNITED STATES 2855
UNITED ARAB EMIRATES 1916
UNITED KINGDOM 1307
AUSTRALIA  994
CANADA     978
SINGAPORE  903
SAUDI ARABIA 845
KUWAIT     627
QATAR      608
GERMANY    378
OMAN       310
NIGERIA    268
HONG KONG  265
1          241
MALAYSIA   185
NEW ZEALAND 172
IRELAND    171
NETHERLANDS 166
BAHRAIN    164
JAPAN      114
THAILAND   81
SWEDEN     77
BELGIUM    76
ITALY      61
Marshall Islands 54
DENMARK    54
INDONESIA  54
KENYA      49
UGANDA     48
CHINA      47
ZAMBIA     47
POLAND     46
FRANCE     45
SPAIN      43
REPUBLIC OF KOREA 42
CYPRUS     42
RUSSIAN FEDERATION 37
SWITZERLAND 36
Philippines 36
VIETNAM    36
FINLAND    35
Democratic Republic Congo 32
TURKEY     31
SOUTH AFRICA 30
PORTUGAL   29
MALTA      26
ISRAEL     25
ANGOLA     25
MOZAMBIQUE 24
IRAQ       24
GREECE     24
NORWAY     23
TANZANIA UNITED REPUBLIC 21
LIBERIA    18
LATVIA     18
MEXICO     18
SRILANKA   18
MALDIVES   18
GABON      18
TAIWAN (REPUBLIC OF CH

In [ ]:
# Count the unique values in column 'v10'
unique_values_counts_v10 = cdna_df_cleaned_2['v10'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v10.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v10.sum()}")


IN         1012277
356        85174
60         792
84         480
29         402
90         348
20         312
560        198
54         192
601        156
443        150
34         126
400        120
411        90
70         84
365        78
348        78
Y51        66
80         66
50         66
271        66
600        66
364        60
10         60
226        54
269        48
30         48
380        42
367        42
81         42
362        42
395        36
96         36
874        30
452        30
68         30
700        30
361        30
820        24
390        24
11D        24
530        24
363        24
Z29        24
440        24
110        24
QA         24
Y55        24
SA         24
Z09        24
970        24
xxxxx      18
291        18
089        18
Y74        18
A60        18
360        18
Z03        18
857        18
A01        18
302        18
Y96        18
AE         18
GB         18
903        18
101        18
39         18
826        18
812        18
228        12
2

In [ ]:
# Count the unique values in column 'v15'
unique_values_counts_v15 = cdna_df_cleaned_2['v15'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v15.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v15.sum()}")


Resident Individuals 526370
Affluent   58182
Mass Affluent 2125
FIRST Millennia 2051
LOS        1923
FIRST Select 1808
FIRST Wealth 1059
FIRST Power Plus 755
FIRST Classic 751
FIRST SWYP 265
0001       262
Club Vistara 197
FIRST Power 188
0188       155
Proprietary concern 144
LIC Select Credit Card 99
NOTAPPLICABLE 96
FIRST WOW! 60
Private Corporates- Non-Financial 54
Non Resident Individuals 54
0187       53
LIC Classic Credit Card 51
FIRST Digital Credit Card 51
Mayura     36
0147       18
Partnership 18
Ashva      15
INDIVIDUAL SALARIED 11
CORPORATE  6
0189       2
FIRST Family 2
0132       2
FIRST EARN Credit Card 1
Total      596864


In [ ]:
# Count the unique values in column 'v30'
unique_values_counts_v30 = cdna_df_cleaned_2['v30'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v30.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v30.sum()}")


95         416293
01         330594
11         136392
02         104279
03         46868
04         40206
15         27711
06         26973
false      19089
05         16033
10         15379
07         12249
16         6758
75         6411
17         2381
52         2112
93         772
99         689
95.0       654
51         644
60         635
1.0        565
33         545
36         485
85         463
70         315
28         279
11.0       184
2.0        181
32         168
74         162
ZZ         125
94         118
34         113
27         96
39         92
91         90
4.0        89
25         84
29         83
18         82
50         75
09         70
20         66
55         60
21         54
3.0        52
31         49
41         48
15.0       47
63         46
5.0        42
45         42
22         36
24         36
65         34
10.0       32
6.0        30
80         25
19         24
48         23
7.0        22
92         18
26         18
37         17
74.0       14
23        

In [ ]:
# Count the unique values in column 'v31'
unique_values_counts_v31 = cdna_df_cleaned_2['v31'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v31.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v31.sum()}")


IN         1091956
356        127915
US         2856
AE         1916
GB         1310
AU         994
CA         978
SG         904
SA         846
KW         627
QA         608
DE         378
OM         310
NG         269
HK         265
MY         185
NZ         172
IE         171
NL         166
BH         164
JP         114
TH         81
SE         77
BE         76
IT         61
MH         54
DK         54
ID         54
KE         49
UG         48
ZM         47
CN         47
PL         46
FR         45
ES         43
KR         42
CY         42
RU         37
CH         36
PH         36
VN         36
FI         35
CD         32
TR         31
ZA         30
PT         29
MT         26
IL         25
AO         25
GR         24
IQ         24
MZ         24
NO         23
TZ         21
LV         18
LR         18
MX         18
BM         18
TW         18
LK         18
MV         18
LU         18
GA         18
KY         18
JO         16
CI         12
HU         12
SL         12
AT         12
TG 

In [ ]:
# Ensure the dataset is loaded and column v31 exists
if 'v31' in cdna_df_cleaned_2.columns:
    unique_count_v31 = cdna_df_cleaned_2['v31'].nunique()
    print(f"Number of unique values in column 'v31': {unique_count_v31}")
else:
    print("Column 'v31' does not exist in the dataset.")


Number of unique values in column 'v31': 101


In [ ]:
# Count the unique values in column 'v37'
unique_values_counts_v37 = cdna_df_cleaned_2['v37'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v37.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v37.sum()}")


MAHARASHTRA 197967
GUJARAT    109325
UTTAR PRADESH 102813
KARNATAKA  101356
TAMIL NADU 92483
ANDHRA PRADESH 69800
WEST BENGAL 66935
TELANGANA  66029
RAJASTHAN  57131
DELHI      52285
HARYANA    47525
NCT OF DELHI 45207
MADHYA PRADESH 43064
PUNJAB     30796
KERALA     27068
BIHAR      26833
ODISHA     17725
CHHATTISGARH 15366
JHARKHAND  14122
ORISSA     13540
ASSAM      9583
UTTARAKHAND 5278
27         5237
CHANDIGARH 4473
07         4175
24         3695
GOA        2832
19         2475
HIMACHAL PRADESH 2379
09         2343
JAMMU and KASHMIR 2340
UTTARANCHAL 2324
29         2055
36         1616
08         1541
33         1383
37         1239
06         1212
TRIPURA    1191
DUBAI      1168
23         976
SINGAPORE  904
PONDICHERRY 759
JAMMU AND KASHMIR 730
MEGHALAYA  642
21         625
MANIPUR    625
32         610
ONTARIO    588
KUWAIT     580
NAGALAND   571
03         559
Saudi Arabia 527
PUDUCHERRY 498
California 471
ABU DHABI  443
10         423
20         416
Victoria   386
NEW SOUTH

In [ ]:
### Dropping columns v9, v10, v15, v30, v31, v37


# Drop the specified columns and store the result in a new dataset
columns_to_drop = ['v9', 'v10', 'v15', 'v30', 'v31', 'v37']
cdna_df_cleaned_3 = cdna_df_cleaned_2.drop(columns=columns_to_drop)

# Verify the new dataset
print(cdna_df_cleaned_3.head())


                                       CUSTOMER_CODE    v5           v6  \
0  ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...  99.0        THANE   
1  6e8e3227297409f3f33578400302825263cadc2ed0d1a0...  99.0         Pune   
2  1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...  99.0      BHIWANI   
3  06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...  99.0  CHIKMAGALUR   
4  0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...  99.0        NASIK   

                        v11  v24        v27                     v29   v32  \
0  2018-08-25T00:00:00.000Z  0.0    MARRIED                SALARIED   NaN   
1  2018-08-03T14:12:16.000Z  NaN    Married  Self Employed Business  60.0   
2  2021-10-08T00:00:00.000Z  0.0    MARRIED                     NaN   NaN   
3  2024-03-18T00:00:00.000Z  NaN  Unmarried                Salaried  84.0   
4  2024-02-26T00:00:00.000Z  NaN  Unmarried               Homemaker  60.0   

                                                 v33      v34  ... v295 v296  \
0  c7e

In [ ]:
# Count the unique values in column 'v63'
unique_values_counts_v63 = cdna_df_cleaned_3['v63'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v63.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v63.sum()}")


100001 to 5L 286050
5L to 10L  86388
0 to 1L    52674
10L to 25L 45162
50L to 3Crore 21564
25L to 50L 20448
0          18426
3Crore to 5Crore 2298
5Crore to 10Crore 1560
10Crore to 15Crore 588
15Crore to 20Crore 210
20Crore to 35Crore 210
35Crore to 50Crore 156
Total      535734


In [ ]:
# Save cdna_df_cleaned_3 to a CSV file
cdna_df_cleaned_3.to_csv('cdna_df_cleaned_3.csv', index=False)

print("Dataset saved as 'cdna_df_cleaned_3.csv'")


Dataset saved as 'cdna_df_cleaned_3.csv'


In [ ]:
# Load the CSV file into a DataFrame
cdna_df_cleaned_3 = pd.read_csv('cdna_df_cleaned_3.csv')

Column v63: Contained Income range in string format. Mapped the range strings to numeric values (mean of range)  for encoding. Filled missing values with median.

In [ ]:
import pandas as pd

# Example mapping of range strings to numeric values for encoding
range_mapping = {
    '0': 0,
    '0 to 1L': 0.5,
    '100001 to 5L': 2.5,
    '5L to 10L': 7.5,
    '10L to 25L': 17.5,
    '25L to 50L': 37.5,
    '50L to 3Crore': 175,
    '3Crore to 5Crore': 400,
    '5Crore to 10Crore': 750,
    '10Crore to 15Crore': 1250,
    '15Crore to 20Crore': 1750,
    '20Crore to 35Crore': 2750,
    '35Crore to 50Crore': 4250,
}

# 1. Map the string values in 'v63' to numeric values
cdna_df_cleaned_3['v63_encoded'] = cdna_df_cleaned_3['v63'].map(range_mapping)

# 2. Calculate the median for the encoded numeric column
median_v63_encoded = cdna_df_cleaned_3['v63_encoded'].median()

# 3. Fill missing values in the encoded column with the median value
cdna_df_cleaned_3['v63_encoded'] = cdna_df_cleaned_3['v63_encoded'].fillna(median_v63_encoded)

# 4. Fill missing values in the original 'v63' column with the calculated median of the encoded values
median_v63 = cdna_df_cleaned_3['v63_encoded'].median()  # Median of the encoded column
cdna_df_cleaned_3['v63'] = cdna_df_cleaned_3['v63'].map(range_mapping).fillna(median_v63)

# Validate the results
print("Cleaned Dataset: Missing values in 'v63':", cdna_df_cleaned_3['v63'].isna().sum())
print("Cleaned Dataset: Missing values in 'v63_encoded':", cdna_df_cleaned_3['v63_encoded'].isna().sum())


Cleaned Dataset: Missing values in 'v63': 0
Cleaned Dataset: Missing values in 'v63_encoded': 0


In [ ]:
# Printing the first few rows of the 'v63' column and its datatype
v63_head = cdna_df_cleaned_3['v63'].head(10)
v63_dtype = cdna_df_cleaned_3['v63'].dtype

(v63_head, v63_dtype)


(0     2.5
 1    37.5
 2     2.5
 3     2.5
 4     2.5
 5     2.5
 6     2.5
 7     2.5
 8     2.5
 9     2.5
 Name: v63, dtype: float64,
 dtype('float64'))

In [ ]:
# Count the unique values in column 'v229'
unique_values_counts_v229 = cdna_df_cleaned_3['v229'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v229.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v229.sum()}")


22101.0    162914
43552.0    146598
22101      92911
22102.0    87680
43552      82680
22102      51101
43554.0    22641
21101.0    22427
43551.0    21479
22106.0    17900
43301.0    17867
21101      13641
43501.0    13232
43554      12683
43553.0    12016
43551      11818
43512.0    11534
43301      10391
22106      10332
43501      7799
43553      6660
43512      6594
23201.0    6038
22107.0    5977
22116.0    4697
43302.0    4509
22125.0    3973
23201      3552
22107      3516
22108.0    3475
21138.0    3152
43556.0    2822
22124.0    2749
43302      2676
22116      2628
44309.0    2615
22125      2244
22105.0    2220
43513.0    2180
43304.0    2095
22108      2028
21138      1750
43556      1565
44309      1535
22124      1510
43305.0    1247
43513      1238
22105      1224
47371.0    1129
43304      1117
43203.0    1082
21122.0    798
43203      708
43305      706
22103.0    650
47371      578
43802.0    560
22123.0    471
43751.0    414
21122      387
43803.0    372
43202.0    36

In [ ]:
# Count the unique values in column 'v230'
unique_values_counts_v230 = cdna_df_cleaned_3['v230'].value_counts()

# Print unique values and their counts in the specified format
for value, count in unique_values_counts_v230.items():
    print(f"{value:<10} {count}")

# Print the total
print(f"{'Total':<10} {unique_values_counts_v230.sum()}")


22101      218986
43552      134461
22102      113113
43553      76028
23201      51776
43554      42292
43551      39613
21101      30736
43512      29839
43301      29530
43501      23066
22106      19453
23217      12235
43302      8259
22107      7295
23222      6229
21138      4816
22108      4666
Nippon India Small Cap Fund(G) 4337
22125      4320
43203      4280
44309      4082
43513      3932
23231      3814
22116      3146
22105      3098
43304      3035
43556      2770
47371      2241
43305      2199
23205      2079
ICICI Pru Value Discovery Fund(G) 1902
22126      1823
22124      1590
43803      1464
43112      1399
43802      1326
23203      1203
ICICI Pru Balanced Advantage Fund(G) 1181
HDFC Mid-Cap Opportunities Fund(G) 1032
21122      974
23218      906
43664      771
43751      763
43402      706
22103      691
43202      687
47501      644
47201      606
43654      586
43208      525
23224      522
47341      466
Axis Bluechip Fund-Reg(G) 453
43651      450
ICICI Pru B

In [ ]:
# Drop columns v229 and v230 from the dataset
cleaned_cdna_df = cdna_df_cleaned_3.drop(columns=['v229', 'v230'])

# Optionally, print the updated dataset or its shape to confirm the columns are dropped
print(cleaned_cdna_df.shape)  # Print the number of rows and columns after dropping


(1285402, 164)


In [ ]:
# Save the cleaned dataset to a CSV file
cleaned_cdna_df.to_csv('cleaned_cdna_df.csv', index=False)  # index=False ensures that the row index is not written

print("Dataset saved as cleaned_cdna_df.csv")


Dataset saved as cleaned_cdna_df.csv


#### Numerical columns with missing values

 For numerical columns with missing values, the most suitable strategy was to replace the missing values with the **median** of each column.

In [ ]:
cleaned_cdna_df=pd.read_csv("cleaned_cdna_df.csv")

In [ ]:
import pandas as pd

# Load your dataset (uncomment and replace with the correct path)
# cleaned_cdna_df = pd.read_csv('path_to_your_dataset.csv')

# Identify numerical columns
numerical_columns = cleaned_cdna_df.select_dtypes(include=['number']).columns

# Check for columns with NaN values
numerical_columns_with_nan = [col for col in numerical_columns if cleaned_cdna_df[col].isna().any()]

# Print the column names that have numerical data and missing values
print(numerical_columns_with_nan)


['v24', 'v32', 'v44', 'v65', 'v77', 'v79', 'v80', 'v84', 'v128', 'v129', 'v130', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v224', 'v225', 'v226', 'v227', 'v228', 'v238', 'v239', 'v240', 'v241', 'v243', 'v244', 'v252', 'v253', 'v254', 'v255', 'v257', 'v258', 'v259', 'v260', 'v261', 'v262', 'v263', 'v264', 'v265', 'v266', 'v267', 'v268', 'v269', 'v270', 'age_mean']


In [ ]:
len(numerical_columns_with_nan)

48

In [ ]:
import pandas as pd

# Load your dataset (uncomment and replace with the correct path)
# cleaned_cdna_df = pd.read_csv('path_to_your_dataset.csv')

# Identify numerical columns
numerical_columns = cleaned_cdna_df.select_dtypes(include=['number']).columns

# Check for columns with NaN values
numerical_columns_with_nan = [col for col in numerical_columns if cleaned_cdna_df[col].isna().any()]

# Create a new DataFrame for the updated data
nume_cdna = cleaned_cdna_df.copy()

# Replace NaN values with the median of each column in numerical_columns_with_nan
for col in numerical_columns_with_nan:
    median_value = nume_cdna[col].median()  # Calculate the median of the column
    nume_cdna[col] = nume_cdna[col].fillna(median_value)  # Replace NaN with median

# Verify the changes
print("Updated dataset with NaN values replaced by medians:")
print(nume_cdna.head())

# Optional: Save the new dataset to a CSV file
# nume_cdna.to_csv('path_to_save_new_dataset.csv', index=False)


Updated dataset with NaN values replaced by medians:
                                       CUSTOMER_CODE    v5           v6  \
0  ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...  99.0        THANE   
1  6e8e3227297409f3f33578400302825263cadc2ed0d1a0...  99.0         Pune   
2  1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...  99.0      BHIWANI   
3  06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...  99.0  CHIKMAGALUR   
4  0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...  99.0        NASIK   

                        v11  v24        v27                     v29   v32  \
0  2018-08-25T00:00:00.000Z  0.0    MARRIED                SALARIED  50.0   
1  2018-08-03T14:12:16.000Z  0.0    Married  Self Employed Business  60.0   
2  2021-10-08T00:00:00.000Z  0.0    MARRIED                     NaN  50.0   
3  2024-03-18T00:00:00.000Z  0.0  Unmarried                Salaried  84.0   
4  2024-02-26T00:00:00.000Z  0.0  Unmarried               Homemaker  60.0   

                                 

#### Categorical with NaN values

In [ ]:
import pandas as pd

# Identify categorical columns in nume_cdna
categorical_columns = nume_cdna.select_dtypes(include=['object']).columns

# Check for columns with NaN values
categorical_columns_with_nan = [col for col in categorical_columns if nume_cdna[col].isna().any()]

# Print the column names and count of categorical columns with NaN values
print("Categorical columns with NaN values:")
print(categorical_columns_with_nan)

# Print the count of categorical columns
print("\nCount of categorical columns with NaN values:", len(categorical_columns_with_nan))


Categorical columns with NaN values:
['v6', 'v11', 'v27', 'v29', 'v33', 'v34', 'v35', 'v42', 'v43', 'v54', 'v55', 'v56', 'v60', 'v66', 'v68', 'v69', 'v71', 'v73', 'v74', 'v81', 'v99', 'v101', 'v102', 'v103', 'v271', 'v272', 'v273', 'v274', 'v275', 'v276', 'v277', 'v278', 'v279', 'v280', 'v281', 'v282', 'v283', 'v284', 'v285', 'v286']

Count of categorical columns with NaN values: 40


**Dropped columns v6, v11, v27, v29, v33, v34, v37, v42, v56, v60, v66, v68, v69, v71, v73, v74, v81, v102, v103, v272, v273, v274, v275, v276, v277, v280, v281, v282, v283, v284, v285 due to reasons like data redundancy, too many categories which could not be encoded etc.**

In [ ]:
### v6,v11,v33,v34,v37,v56,v60,v66,v68,v69,v71,v73,v74,v81,v102,v103,v272,v273,v274,v275,v276,v277,v280,v281,v282,v283,v284,v285
# List of columns to drop
columns_to_drop = ['v6', 'v11', 'v33', 'v34', 'v56', 'v60', 'v66', 'v68', 'v69', 'v71', 'v73', 'v74',
                   'v81', 'v102', 'v103', 'v272', 'v273', 'v274', 'v275', 'v276', 'v277', 'v280', 'v281', 'v282',
                   'v283', 'v284', 'v285']

# Drop the specified columns and save the result in a new DataFrame
nume_cdna_dropped = nume_cdna.drop(columns=columns_to_drop)

# Print the updated DataFrame
print(nume_cdna_dropped)


                                             CUSTOMER_CODE    v5  v24  \
0        ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...  99.0  0.0   
1        6e8e3227297409f3f33578400302825263cadc2ed0d1a0...  99.0  0.0   
2        1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...  99.0  0.0   
3        06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...  99.0  0.0   
4        0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...  99.0  0.0   
...                                                    ...   ...  ...   
1285397  208d6130ed06bfa28102a553b800db738f348aee78ceed...  99.0  1.0   
1285398  7ef63f0b16fa55ef0f2df11483014c166462c0433a2228...  99.0  0.0   
1285399  36de75588001e2282d38aee12b87919c7d0a359af49b5d...  99.0  0.0   
1285400  0beadfd5b2e548d6e4c5aaa730b7766b21ad5e5e0d88c3...  99.0  0.0   
1285401  6b3b00bad12064c850bcad5e45038b0992944e6c652bfd...  99.0  0.0   

               v27                     v29   v32            v35       v36  \
0          MARRIED                SALARIED  50

In [ ]:
# Identify categorical columns
categorical_columns = nume_cdna_dropped.select_dtypes(include=['object'])

# Find columns in the categorical columns that have NaN values
categorical_columns_with_nan = categorical_columns.columns[categorical_columns.isna().any()].tolist()

# Print the categorical columns with NaN values and their count
print("Categorical columns with NaN values:", categorical_columns_with_nan)
print("Column count:", len(categorical_columns_with_nan))


Categorical columns with NaN values: ['v27', 'v29', 'v35', 'v42', 'v43', 'v54', 'v55', 'v99', 'v101', 'v271', 'v278', 'v279', 'v286']
Column count: 13


In [ ]:
# Drop the specified columns from the cleaned dataset
nume_cdna_dropped= nume_cdna_dropped.drop(columns=['v35', 'v271', 'v279'])

# Display the remaining columns and their count
nume_cdna_dropped.columns.tolist(), len(nume_cdna_dropped.columns)


(['CUSTOMER_CODE',
  'v5',
  'v24',
  'v27',
  'v29',
  'v32',
  'v36',
  'v39',
  'v40',
  'v41',
  'v42',
  'v43',
  'v44',
  'v45',
  'v49',
  'v50',
  'v51',
  'v52',
  'v53',
  'v54',
  'v55',
  'v63',
  'v65',
  'v77',
  'v79',
  'v80',
  'v82',
  'v83',
  'v84',
  'v86',
  'v87',
  'v88',
  'v89',
  'v90',
  'v91',
  'v92',
  'v93',
  'v94',
  'v95',
  'v96',
  'v97',
  'v98',
  'v99',
  'v100',
  'v101',
  'v104',
  'v105',
  'v106',
  'v107',
  'v108',
  'v109',
  'v128',
  'v129',
  'v130',
  'v131',
  'v132',
  'v133',
  'v137',
  'v138',
  'v139',
  'v143',
  'v144',
  'v145',
  'v161',
  'v162',
  'v183',
  'v184',
  'v185',
  'v186',
  'v203',
  'v204',
  'v205',
  'v206',
  'v207',
  'v208',
  'v209',
  'v224',
  'v225',
  'v226',
  'v227',
  'v228',
  'v231',
  'v232',
  'v233',
  'v234',
  'v235',
  'v236',
  'v237',
  'v238',
  'v239',
  'v240',
  'v241',
  'v243',
  'v244',
  'v252',
  'v253',
  'v254',
  'v255',
  'v256',
  'v257',
  'v258',
  'v259',
  'v260',
  'v

In [ ]:
# Find the columns which have categorical data in nume_cdna_dropped
categorical_columns = nume_cdna_dropped.select_dtypes(include=['object']).columns.tolist()
categorical_columns


['CUSTOMER_CODE',
 'v27',
 'v29',
 'v36',
 'v42',
 'v43',
 'v54',
 'v55',
 'v99',
 'v101',
 'v278',
 'v286',
 'batch_date']

In [ ]:
nume_cdna_dropped[categorical_columns].isnull().sum()

CUSTOMER_CODE         0
v27               60259
v29              234294
v36                   0
v42              587777
v43               61223
v54               45203
v55              766354
v99              650317
v101              34374
v278             480591
v286             480591
batch_date            0
dtype: int64

In [ ]:
unique_values = {col: nume_cdna_dropped[col].unique() for col in categorical_columns}

# Print the unique values column-wise
for column, values in unique_values.items():
    print(f"Unique values in '{column}': {values}")

Unique values in 'CUSTOMER_CODE': ['ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a984784be6f123024ea50'
 '6e8e3227297409f3f33578400302825263cadc2ed0d1a06e9b9d0f50d94ba216'
 '1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7aa8b06e89c46fc754f' ...
 'be6dc2009f5132fa514dc2a0cdc88ea333c870516c5333df32e44cf046393078'
 '33a34e5263bdff0a66325ed4591f74bca48e1ad6218d2415acf1fdce706d35ab'
 'abba01a175c6bfc1f5908e1511561eafa78f35d4f389af72470e6889fb85d934']
Unique values in 'v27': ['MARRIED' 'Married' 'Unmarried' 'UNMARRIED' 'S' nan 'M' 'OTHER' 'Widow'
 'SINGLE' 'WIDOW' 'Single']
Unique values in 'v29': ['SALARIED' 'Self Employed Business' nan 'Salaried' 'Homemaker'
 'SELF EMPLOYED' 'Salaried-Private' 'OTHER' 'Student'
 'Selp employed profesional' 'Salarid-Public' 'Retired' 'Salarid-Govt'
 'OTHERSSTUDENT' 'Self Employed' 'Self Employed Professional-Doctor'
 'Farmer' 'CORPORATE' 'Unemployed' 'Self Employed Professional-CA'
 'NA - Minor' 'NORMAL' 'INDIVIDUAL SELF EMPLOYED' 'INDIVIDUAL SALARIED'
 'SALAR

In [ ]:
nume_cdna_dropped[['v27','v43']].isnull().sum()

v27    60259
v43    61223
dtype: int64

In [ ]:
nume_cdna_dropped= nume_cdna_dropped.drop(columns=['v29', 'v42', 'v27'])
nume_cdna_dropped.shape

(1285402, 131)

In [ ]:
nume_cdna_dropped = nume_cdna_dropped.reset_index(drop=True)
nume_cdna_dropped.head()

,CUSTOMER_CODE,v5,v24,v32,v36,v39,v40,v41,v43,v44,...,v296,v297,v298,v299,v300,v301,v302,batch_date,age_mean,v63_encoded
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,99.0,0.0,50.0,MDM,False,False,False,MARRIED,4.0,...,204,150,73,271,164,170,246,2024-09-27,52.0,2.5
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,99.0,0.0,60.0,MDM,False,False,False,MARRIED,4.0,...,204,150,73,271,164,170,246,2024-09-27,37.0,37.5
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,99.0,0.0,50.0,MDM,False,False,False,MARRIED,4.0,...,204,150,73,271,164,170,246,2024-09-27,22.0,2.5
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,99.0,0.0,84.0,MDM,False,False,False,UNMARRIED,4.0,...,204,150,73,271,164,170,246,2024-09-27,22.0,2.5
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,99.0,0.0,60.0,MDM,False,False,False,UNMARRIED,4.0,...,204,150,73,271,164,170,246,2024-09-27,32.0,2.5


### Loading the target Column and joining it with the cdna datset

We are joining before encoding the categorical columns so as to save computational time since the number of columns in target dataset is almost half.

In [ ]:
target_dataset=pd.read_csv('target_median.csv')

In [ ]:
target_dataset.head()

,batch_date_history,customer_code,target_slot
0,2024-05-31,00006eb623a7a335fb6fa7334939cca08c75e80e44e404...,19
1,2024-05-31,0000b844339117db33d8dcc6ca4e97a8c989a918c8cf01...,9
2,2024-05-31,00021bdab2ed517d593cce06bffe6a002d6ad363048e61...,14
3,2024-05-31,00064e0390f08b415acc060257dd29bfed915bb4ff4fef...,14
4,2024-05-31,0007274f2cbe775d59dadbbde24d10f935f466af1ce03a...,14


In [ ]:
merged_dataset = pd.merge(
    left=target_dataset,
    right=nume_cdna_dropped,
    how='inner',  # Using 'inner' to include only matching rows
    left_on=['customer_code', 'batch_date_history'],
    right_on=['CUSTOMER_CODE', 'batch_date']
)

In [ ]:
merged_dataset.shape

(645053, 134)

In [ ]:
merged_dataset.head()

,batch_date_history,customer_code,target_slot,CUSTOMER_CODE,v5,v24,v32,v36,v39,v40,...,v296,v297,v298,v299,v300,v301,v302,batch_date,age_mean,v63_encoded
0,2024-05-31,00006eb623a7a335fb6fa7334939cca08c75e80e44e404...,19,00006eb623a7a335fb6fa7334939cca08c75e80e44e404...,99.0,0.0,7.0,MDM,False,False,...,85,31,318,152,45,51,127,2024-05-31,42.0,2.5
1,2024-05-31,0000b844339117db33d8dcc6ca4e97a8c989a918c8cf01...,9,0000b844339117db33d8dcc6ca4e97a8c989a918c8cf01...,99.0,0.0,20.0,MDM,False,False,...,85,31,318,152,45,51,127,2024-05-31,22.0,2.5
2,2024-05-31,00021bdab2ed517d593cce06bffe6a002d6ad363048e61...,14,00021bdab2ed517d593cce06bffe6a002d6ad363048e61...,99.0,0.0,50.0,FINNONE,False,False,...,85,31,318,152,45,51,127,2024-05-31,32.0,2.5
3,2024-05-31,00064e0390f08b415acc060257dd29bfed915bb4ff4fef...,14,00064e0390f08b415acc060257dd29bfed915bb4ff4fef...,99.0,0.0,50.0,MDM,False,False,...,85,31,318,152,45,51,127,2024-05-31,37.0,2.5
4,2024-05-31,0007274f2cbe775d59dadbbde24d10f935f466af1ce03a...,14,0007274f2cbe775d59dadbbde24d10f935f466af1ce03a...,99.0,0.0,50.0,MDM,False,False,...,85,31,318,152,45,51,127,2024-05-31,22.0,2.5


In [ ]:
merged_dataset=merged_dataset.drop(columns=['batch_date_history','customer_code'])

In [ ]:
merged_dataset.head()

,target_slot,CUSTOMER_CODE,v5,v24,v32,v36,v39,v40,v41,v43,...,v296,v297,v298,v299,v300,v301,v302,batch_date,age_mean,v63_encoded
0,19,00006eb623a7a335fb6fa7334939cca08c75e80e44e404...,99.0,0.0,7.0,MDM,False,False,False,MARRIED,...,85,31,318,152,45,51,127,2024-05-31,42.0,2.5
1,9,0000b844339117db33d8dcc6ca4e97a8c989a918c8cf01...,99.0,0.0,20.0,MDM,False,False,False,UNMARRIED,...,85,31,318,152,45,51,127,2024-05-31,22.0,2.5
2,14,00021bdab2ed517d593cce06bffe6a002d6ad363048e61...,99.0,0.0,50.0,FINNONE,False,False,False,SINGLE,...,85,31,318,152,45,51,127,2024-05-31,32.0,2.5
3,14,00064e0390f08b415acc060257dd29bfed915bb4ff4fef...,99.0,0.0,50.0,MDM,False,False,False,UNMARRIED,...,85,31,318,152,45,51,127,2024-05-31,37.0,2.5
4,14,0007274f2cbe775d59dadbbde24d10f935f466af1ce03a...,99.0,0.0,50.0,MDM,False,False,False,UNMARRIED,...,85,31,318,152,45,51,127,2024-05-31,22.0,2.5


# CLEANING AND CREATING THE FINAL DATASET

### One-Hot Encoding the Categorical columns left one by one

In [ ]:
categorical_columns_merged = merged_dataset.select_dtypes(include=['object']).columns.tolist()
categorical_columns_merged

['CUSTOMER_CODE',
 'v36',
 'v43',
 'v54',
 'v55',
 'v99',
 'v101',
 'v278',
 'v286',
 'batch_date']

#### V36

In [ ]:
merged_dataset['v36'].unique()

array(['MDM', 'FINNONE', 'MDM_CORP', 'CUSTOMER'], dtype=object)

In [ ]:
merged_df36 = pd.get_dummies(merged_dataset, columns=['v36'], drop_first=True)

In [ ]:
merged_df36.columns

Index(['target_slot', 'CUSTOMER_CODE', 'v5', 'v24', 'v32', 'v39', 'v40', 'v41',
       'v43', 'v44',
       ...
       'v299', 'v300', 'v301', 'v302', 'batch_date', 'age_mean', 'v63_encoded',
       'v36_FINNONE', 'v36_MDM', 'v36_MDM_CORP'],
      dtype='object', length=134)

#### V43

In [ ]:
merged_df36['v43'].unique()

array(['MARRIED', 'UNMARRIED', 'SINGLE', nan, 'OTHERS'], dtype=object)

In [ ]:
most_frequent_count = merged_df36['v43'].value_counts()
most_frequent_count

v43
MARRIED      309331
UNMARRIED    290697
SINGLE         7258
OTHERS           10
Name: count, dtype: int64

In [ ]:
merged_df36['v43'].isnull().sum()

37757

In [ ]:
merged_df36['v43'] = merged_df36['v43'].fillna('OTHERS')

In [ ]:
merged_df43 = pd.get_dummies(merged_df36, columns=['v43'], drop_first=True)

#### V54

In [ ]:
merged_df43['v54'].unique()

array(['M', 'Female', 'MALE', 'Male', 'F', 'FEMALE', nan, 'THIRD GENDER',
       'C', 'U', 'OTHERS', 'O'], dtype=object)

In [ ]:
most_frequent_count = merged_df43['v54'].value_counts()
most_frequent_count

v54
Male            262320
MALE            167137
Female           78854
M                75287
FEMALE           26791
F                 8045
THIRD GENDER        46
C                   35
OTHERS              16
U                   10
O                    1
Name: count, dtype: int64

In [ ]:
merged_df43['v54'].isnull().sum()

26511

In [ ]:
merged_df43['v54'] = merged_df43['v54'].fillna('Male')

In [ ]:
merged_df43['v54'] = merged_df43['v54'].replace(['MALE', 'Male', 'M'], 'Male')
merged_df43['v54'] = merged_df43['v54'].replace(['FEMALE', 'Female', 'F'], 'Female')
merged_df43['v54'] = merged_df43['v54'].replace(['C', 'OTHERS', 'THIRD GENDER','U','O'], 'Others')

In [ ]:
most_frequent_count = merged_df43['v54'].value_counts()
most_frequent_count

v54
Male      531255
Female    113690
Others       108
Name: count, dtype: int64

In [ ]:
merged_df54 = pd.get_dummies(merged_df43, columns=['v54'], drop_first=False)

In [ ]:
merged_df54.columns

Index(['target_slot', 'CUSTOMER_CODE', 'v5', 'v24', 'v32', 'v39', 'v40', 'v41',
       'v44', 'v45',
       ...
       'v63_encoded', 'v36_FINNONE', 'v36_MDM', 'v36_MDM_CORP', 'v43_OTHERS',
       'v43_SINGLE', 'v43_UNMARRIED', 'v54_Female', 'v54_Male', 'v54_Others'],
      dtype='object', length=138)

In [ ]:
merged_df54=merged_df54.drop(columns=['v54_Others'])

#### V55

In [ ]:
most_frequent_count = merged_df54['v55'].value_counts()
most_frequent_count

v55
[GMAIL.COM]            272312
[YAHOO.COM]              4065
[HOTMAIL.COM]            2123
[YAHOO.CO.IN]            2012
[OUTLOOK.COM]             691
                        ...  
[KNOWBAL.COM.AU]            1
[GMQIL.COM]                 1
[HETERO.COM]                1
[METROSOLUTIONS.IN]         1
[BRIVAS.COM]                1
Name: count, Length: 1167, dtype: int64

In [ ]:
merged_df54=merged_df54.drop(columns=['v55'])

#### V99

In [ ]:
merged_df54['v99'].unique()

array([nan, 'ios', 'and'], dtype=object)

In [ ]:
most_frequent_count = merged_df54['v99'].value_counts()
most_frequent_count

v99
and    239727
ios    119061
Name: count, dtype: int64

In [ ]:
merged_df54['v99'] = merged_df54['v99'].fillna('Nan')

In [ ]:
merged_df99 = pd.get_dummies(merged_df54, columns=['v99'], drop_first=True)

In [ ]:
merged_df99.columns

Index(['target_slot', 'CUSTOMER_CODE', 'v5', 'v24', 'v32', 'v39', 'v40', 'v41',
       'v44', 'v45',
       ...
       'v36_FINNONE', 'v36_MDM', 'v36_MDM_CORP', 'v43_OTHERS', 'v43_SINGLE',
       'v43_UNMARRIED', 'v54_Female', 'v54_Male', 'v99_and', 'v99_ios'],
      dtype='object', length=137)

#### V101

In [ ]:
merged_df99['v101'].unique()

array(['Tier 1', nan, 'Tier 5', 'Tier 7', 'Rural', 'Tier 2', 'Tier 6',
       'Tier 8', 'Tier 4', 'Tier 3'], dtype=object)

In [ ]:
most_frequent_count = merged_df99['v101'].value_counts()
most_frequent_count

v101
Tier 1    274965
Rural      75706
Tier 3     53869
Tier 8     53062
Tier 2     47496
Tier 7     43387
Tier 6     37176
Tier 5     27699
Tier 4     15390
Name: count, dtype: int64

In [ ]:
merged_df99['v101'].isnull().sum()

16303

In [ ]:
merged_df99['v101'] = merged_df99['v101'].fillna("Tier 1")

In [ ]:
merged_df101 = pd.get_dummies(merged_df99, columns=['v101'], drop_first=False)

In [ ]:
merged_df101=merged_df101.drop(columns=['v101_Tier 8'])
merged_df101.columns

Index(['target_slot', 'CUSTOMER_CODE', 'v5', 'v24', 'v32', 'v39', 'v40', 'v41',
       'v44', 'v45',
       ...
       'v99_and', 'v99_ios', 'v101_Rural', 'v101_Tier 1', 'v101_Tier 2',
       'v101_Tier 3', 'v101_Tier 4', 'v101_Tier 5', 'v101_Tier 6',
       'v101_Tier 7'],
      dtype='object', length=144)

#### V278

In [ ]:
merged_df101['v278'].unique()

array(['MORNING', nan, 'EVENING'], dtype=object)

In [ ]:
merged_df101['v278'].isnull().sum()

191291

In [ ]:
merged_df101['v278'] = merged_df101['v278'].fillna("Other")

In [ ]:
merged_df278 = pd.get_dummies(merged_df101, columns=['v278'], drop_first=False)

In [ ]:
merged_df278=merged_df278.drop(columns=['v278_Other'])
merged_df278.columns

Index(['target_slot', 'CUSTOMER_CODE', 'v5', 'v24', 'v32', 'v39', 'v40', 'v41',
       'v44', 'v45',
       ...
       'v101_Rural', 'v101_Tier 1', 'v101_Tier 2', 'v101_Tier 3',
       'v101_Tier 4', 'v101_Tier 5', 'v101_Tier 6', 'v101_Tier 7',
       'v278_EVENING', 'v278_MORNING'],
      dtype='object', length=145)

#### V286

In [ ]:
merged_df278['v286'].unique()

array(['WEEKEND', nan, 'WEEKDAY', 'FRIDAY'], dtype=object)

In [ ]:
merged_df278['v286'].isnull().sum()

191291

In [ ]:
most_frequent_count = merged_df278['v286'].value_counts()
most_frequent_count

v286
WEEKEND    359052
WEEKDAY     83258
FRIDAY      11452
Name: count, dtype: int64

In [ ]:
merged_df278['v286'] = merged_df278['v286'].fillna("Other")

In [ ]:
merged_df286 = pd.get_dummies(merged_df278, columns=['v286'], drop_first=False)

In [ ]:
merged_df286=merged_df286.drop(columns=['v286_Other'])
merged_df286.columns

Index(['target_slot', 'CUSTOMER_CODE', 'v5', 'v24', 'v32', 'v39', 'v40', 'v41',
       'v44', 'v45',
       ...
       'v101_Tier 3', 'v101_Tier 4', 'v101_Tier 5', 'v101_Tier 6',
       'v101_Tier 7', 'v278_EVENING', 'v278_MORNING', 'v286_FRIDAY',
       'v286_WEEKDAY', 'v286_WEEKEND'],
      dtype='object', length=147)

In [ ]:
bool_columns = merged_df286.select_dtypes(include='bool').columns
bool_columns

Index(['v39', 'v40', 'v41', 'v45', 'v49', 'v50', 'v51', 'v52', 'v53',
       'v36_FINNONE', 'v36_MDM', 'v36_MDM_CORP', 'v43_OTHERS', 'v43_SINGLE',
       'v43_UNMARRIED', 'v54_Female', 'v54_Male', 'v99_and', 'v99_ios',
       'v101_Rural', 'v101_Tier 1', 'v101_Tier 2', 'v101_Tier 3',
       'v101_Tier 4', 'v101_Tier 5', 'v101_Tier 6', 'v101_Tier 7',
       'v278_EVENING', 'v278_MORNING', 'v286_FRIDAY', 'v286_WEEKDAY',
       'v286_WEEKEND'],
      dtype='object')

In [ ]:
merged_df286[bool_columns] =  merged_df286[bool_columns].astype(int)

In [ ]:
describe_df=pd.DataFrame(merged_df286.describe())
describe_df

,target_slot,v5,v24,v32,v39,v40,v41,v44,v45,v49,...,v101_Tier 3,v101_Tier 4,v101_Tier 5,v101_Tier 6,v101_Tier 7,v278_EVENING,v278_MORNING,v286_FRIDAY,v286_WEEKDAY,v286_WEEKEND
count,645053.000000,645053.000000,645053.000000,645053.000000,645053.0,645053.0,645053.0,645053.000000,645053.000000,645053.000000,...,645053.000000,645053.000000,645053.000000,645053.000000,645053.000000,645053.000000,645053.000000,645053.000000,645053.000000,645053.000000
mean,14.277128,98.766588,0.041170,54.087925,0.0,0.0,0.0,3.687821,0.933968,0.016580,...,0.083511,0.023859,0.042941,0.057632,0.067261,0.056902,0.646547,0.017754,0.129072,0.556624
std,6.920785,3.964724,0.198684,67.667081,0.0,0.0,0.0,0.622331,0.248338,0.127692,...,0.276653,0.152608,0.202724,0.233047,0.250474,0.231656,0.478042,0.132055,0.335279,0.496784
min,1.000000,1.000000,0.000000,1.000000,0.0,0.0,0.0,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,99.000000,0.000000,29.000000,0.0,0.0,0.0,4.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,14.000000,99.000000,0.000000,50.000000,0.0,0.0,0.0,4.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,19.000000,99.000000,0.000000,60.000000,0.0,0.0,0.0,4.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,28.000000,99.000000,1.000000,979.000000,0.0,0.0,0.0,4.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
merged_df286.shape

(645053, 147)

In [ ]:
for x in merged_df286.columns:
    print(x)

target_slot
CUSTOMER_CODE
v5
v24
v32
v39
v40
v41
v44
v45
v49
v50
v51
v52
v53
v63
v65
v77
v79
v80
v82
v83
v84
v86
v87
v88
v89
v90
v91
v92
v93
v94
v95
v96
v97
v98
v100
v104
v105
v106
v107
v108
v109
v128
v129
v130
v131
v132
v133
v137
v138
v139
v143
v144
v145
v161
v162
v183
v184
v185
v186
v203
v204
v205
v206
v207
v208
v209
v224
v225
v226
v227
v228
v231
v232
v233
v234
v235
v236
v237
v238
v239
v240
v241
v243
v244
v252
v253
v254
v255
v256
v257
v258
v259
v260
v261
v262
v263
v264
v265
v266
v267
v268
v269
v270
v287
v288
v289
v290
v291
v292
v293
v294
v295
v296
v297
v298
v299
v300
v301
v302
batch_date
age_mean
v63_encoded
v36_FINNONE
v36_MDM
v36_MDM_CORP
v43_OTHERS
v43_SINGLE
v43_UNMARRIED
v54_Female
v54_Male
v99_and
v99_ios
v101_Rural
v101_Tier 1
v101_Tier 2
v101_Tier 3
v101_Tier 4
v101_Tier 5
v101_Tier 6
v101_Tier 7
v278_EVENING
v278_MORNING
v286_FRIDAY
v286_WEEKDAY
v286_WEEKEND


In [ ]:
null_count=pd.DataFrame(merged_df286.isnull().sum())
null_count.columns=['count']
for index, row in null_count.iterrows():
    for col_name in null_count.columns:
        if (row[col_name]!=0):
            print(index,row[col_name])

In [ ]:
for x in merged_df286.columns:
    print(x)

target_slot
CUSTOMER_CODE
v5
v24
v32
v39
v40
v41
v44
v45
v49
v50
v51
v52
v53
v63
v65
v77
v79
v80
v82
v83
v84
v86
v87
v88
v89
v90
v91
v92
v93
v94
v95
v96
v97
v98
v100
v104
v105
v106
v107
v108
v109
v128
v129
v130
v131
v132
v133
v137
v138
v139
v143
v144
v145
v161
v162
v183
v184
v185
v186
v203
v204
v205
v206
v207
v208
v209
v224
v225
v226
v227
v228
v231
v232
v233
v234
v235
v236
v237
v238
v239
v240
v241
v243
v244
v252
v253
v254
v255
v256
v257
v258
v259
v260
v261
v262
v263
v264
v265
v266
v267
v268
v269
v270
v287
v288
v289
v290
v291
v292
v293
v294
v295
v296
v297
v298
v299
v300
v301
v302
batch_date
age_mean
v63_encoded
v36_FINNONE
v36_MDM
v36_MDM_CORP
v43_OTHERS
v43_SINGLE
v43_UNMARRIED
v54_Female
v54_Male
v99_and
v99_ios
v101_Rural
v101_Tier 1
v101_Tier 2
v101_Tier 3
v101_Tier 4
v101_Tier 5
v101_Tier 6
v101_Tier 7
v278_EVENING
v278_MORNING
v286_FRIDAY
v286_WEEKDAY
v286_WEEKEND


In [ ]:
merged_df286.head()

,target_slot,CUSTOMER_CODE,v5,v24,v32,v39,v40,v41,v44,v45,...,v101_Tier 3,v101_Tier 4,v101_Tier 5,v101_Tier 6,v101_Tier 7,v278_EVENING,v278_MORNING,v286_FRIDAY,v286_WEEKDAY,v286_WEEKEND
0,19,00006eb623a7a335fb6fa7334939cca08c75e80e44e404...,99.0,0.0,7.0,0,0,0,4.0,1,...,0,0,0,0,0,0,1,0,0,1
1,9,0000b844339117db33d8dcc6ca4e97a8c989a918c8cf01...,99.0,0.0,20.0,0,0,0,3.0,1,...,0,0,0,0,0,0,1,0,0,1
2,14,00021bdab2ed517d593cce06bffe6a002d6ad363048e61...,99.0,0.0,50.0,0,0,0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
3,14,00064e0390f08b415acc060257dd29bfed915bb4ff4fef...,99.0,0.0,50.0,0,0,0,4.0,0,...,0,0,1,0,0,0,1,0,0,1
4,14,0007274f2cbe775d59dadbbde24d10f935f466af1ce03a...,99.0,0.0,50.0,0,0,0,4.0,1,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
merged_df286.to_csv("merged_dataset_slots.csv",index=False)

# MODEL DEVELOPMENT AND TRAINING

#### Splitting into X and Y

In [ ]:
X = merged_df286.iloc[:, 2:]
y = merged_df286['target_slot']
X=X.drop(columns=['batch_date'])

In [ ]:
X.to_csv("X.csv", index=False)
y.to_csv("y.csv", index=False)

In [ ]:
X=pd.read_csv("X.csv")
y=pd.read_csv("y.csv")

In [ ]:
X=X.drop(columns=['v63'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#### Scaling and Encoding the Data

In [ ]:
scaler = StandardScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

XGBoost num_class parameter expects values from 0 o 27 if num_classes=28 that is why we have encoded the target column in the suitable range.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

### Model Training

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
weights = np.array([class_weights[c] for c in y_train_encoded])

In [ ]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',  # Use 'multi:softmax' for multi-class classification
    num_class=28,              # Specify the number of classes
    eval_metric='mlogloss',    # Multi-class log loss
    use_label_encoder=False,# Avoid unnecessary warnings
    max_depth=10,
    n_estimators=10
)

In [ ]:
# Train the model on the entire training data
xgb_model.fit(
    X_train_scaled,
    y_train_encoded,# Add eval set to monitor progress
    verbose=True,
)

In [ ]:
y_pred_transformed = xgb_model.predict(X_test_scaled)

# Convert predictions back to original labels (1-28)
y_pred = encoder.inverse_transform(y_pred_transformed)

# Convert test labels back to original labels for comparison
y_test_original = encoder.inverse_transform(y_test_encoded)

# Evaluate accuracy
accuracy = accuracy_score(y_test_original, y_pred)
print("Test set accuracy:", accuracy)

Test set accuracy: 0.07321856275821442


In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test_original, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

Mean Absolute Error (MAE): 6.31


In [ ]:
# Predict on a small subset of the test data
sample_X = X_test[:40]
sample_y_pred_transformed = xgb_model.predict(sample_X)
sample_y_pred = encoder.inverse_transform(sample_y_pred_transformed)
sample_y_test = encoder.inverse_transform(y_test[:40])

print("Predictions:", sample_y_pred)
print("Actual labels:", sample_y_test)

Predictions: [ 7  7  6  7 19  7  5  7  7  7  7  7 20  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  6 22  7 12  7 16  7  7  7  7 20]
Actual labels: [15  9 25 11 28 26 25 14 15  3  6 21 19  7  8 15 28  7  9 10  3 16 24  3
 10 12  5 15 22 24 18  9  8  6  5  7 15  8 14  7]


In [ ]:
train_accuracy =xgb_model.score(X_train_scaled, y_train_encoded)
print(f"Training Accuracy: {train_accuracy:.2f}")

Training Accuracy: 0.18


In [ ]:
model = Sequential([
    Dense(512, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dropout(0.3),                                               # Dropout for regularization
    Dense(256, activation='relu'),                              # Second hidden layer
    Dropout(0.3),
    Dense(128, activation='relu'),                              # Third hidden layer
    Dropout(0.3),
    Dense(64, activation='relu'),                               # Fourth hidden layer
    Dropout(0.3),
    Dense(28, activation='softmax')                             # Output layer for 28 classes
])

# Compile the model
model.compile(
    optimizer='adam',                         
    loss='categorical_crossentropy',         
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train_scaled, y_train_encoded,
    validation_data=(X_test, y_test),
    epochs=10,                               
    batch_size=128,                           
    verbose=1                               
)

In [ ]:
y_pred_transformed = model.predict(X_test_scaled)

# Convert predictions back to original labels (1-28)
y_pred = encoder.inverse_transform(y_pred_transformed)

# Convert test labels back to original labels for comparison
y_test_original = encoder.inverse_transform(y_test_encoded)

In [ ]:
mae = mean_absolute_error(y_test_original, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# TESTING DATA ANALYSIS AND PREPARATION

### Loading Data

In [ ]:
test_history=pd.read_csv("test_action_history.csv")
test_cdna=pd.read_csv("test_cdna_data.csv")
test_cdna['CUSTOMER_CODE'].nunique()
test_history['customer_code'].nunique()

64700

In [ ]:
test_cdna['CUSTOMER_CODE'].nunique()

68450

Since test_cdna number of customers is more than test_history customers upon merge customers to whom no mails have been sent will be eliminated.

### Cleaning test_cdna data as done for train_cdna

In [ ]:
if 'v2' in test_cdna.columns:
    test_cdna['age_mean'] = test_cdna['v2'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2 if isinstance(x, list) else np.nan)
    test_cdna.drop('v2', axis=1, inplace=True)
    print("\nProcessed Age Range into Mean:")


Processed Age Range into Mean:


In [ ]:
test_cdna['age_mean']

0        42.0
1        42.0
2        37.0
3        32.0
4        27.0
         ... 
68445    37.0
68446    42.0
68447    22.0
68448    47.0
68449    27.0
Name: age_mean, Length: 68450, dtype: float64

In [ ]:
for col in categorical_cols_90_95:
    # Create a new numerical column based on the categorical column
    new_col = col + "_num"
    test_cdna[new_col] = test_dna[col].apply(lambda x: 1 if pd.notnull(x) else 0)

    # Drop the original categorical column
    test_cdna= test_cdna.drop(columns=[col])

    print(f"Processed and dropped categorical column: {col}")

In [ ]:
test_cdna.columns

Index(['CUSTOMER_CODE', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11',
       ...
       'v295', 'v296', 'v297', 'v298', 'v299', 'v300', 'v301', 'v302',
       'batch_date', 'age_mean'],
      dtype='object', length=303)

In [ ]:
range_mapping = {
    '0': 0,
    '0 to 1L': 0.5,
    '100001 to 5L': 2.5,
    '5L to 10L': 7.5,
    '10L to 25L': 17.5,
    '25L to 50L': 37.5,
    '50L to 3Crore': 175,
    '3Crore to 5Crore': 400,
    '5Crore to 10Crore': 750,
    '10Crore to 15Crore': 1250,
    '15Crore to 20Crore': 1750,
    '20Crore to 35Crore': 2750,
    '35Crore to 50Crore': 4250,
}

# 1. Map the string values in 'v63' to numeric values
test_cdna['v63_encoded'] = test_cdna['v63'].map(range_mapping)

# 2. Calculate the median for the encoded numeric column
median_v63_encoded = test_cdna['v63_encoded'].median()

# 3. Fill missing values in the encoded column with the median value
test_cdna['v63_encoded'] = test_cdna['v63_encoded'].fillna(median_v63_encoded)

# Validate the results
print("Cleaned Dataset: Missing values in 'v63':", test_cdna['v63'].isna().sum())
print("Cleaned Dataset: Missing values in 'v63_encoded':", test_cdna['v63_encoded'].isna().sum())

Cleaned Dataset: Missing values in 'v63': 40732
Cleaned Dataset: Missing values in 'v63_encoded': 0


In [ ]:
test_cdna=test_cdna.drop(columns=['v63'])

In [ ]:
cleaned_cdna_df.drop(columns=['v63'],inplace=True)

In [ ]:
test_cdna=test_cdna[cleaned_cdna_df.columns]

In [ ]:
# Step 1: Convert numerical strings to integers where possible
test_cdna['v5'] = pd.to_numeric(test_cdna['v5'], errors='coerce')

# Step 2: Replace 'ZZ' and NaN with the median of the column
median_v5 = test_cdna['v5'].median()
test_cdna['v5'].fillna(median_v5, inplace=True)


In [ ]:
numerical_columns = test_cdna.select_dtypes(include=['number']).columns
numerical_columns_with_nan = [col for col in numerical_columns if test_cdna[col].isna().any()]

# Replace NaN values with the median of each column in numerical_columns_with_nan
for col in numerical_columns_with_nan:
    median_value = test_cdna[col].median()  # Calculate the median of the column
    test_cdna[col] = test_cdna[col].fillna(median_value)  # Replace NaN with median


In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v36'], drop_first=False)

In [ ]:
test_cdna.columns

Index(['CUSTOMER_CODE', 'v5', 'v6', 'v11', 'v24', 'v27', 'v29', 'v32', 'v33',
       'v34',
       ...
       'v299', 'v300', 'v301', 'v302', 'batch_date', 'age_mean', 'v63_encoded',
       'v36_FINNONE', 'v36_MDM', 'v36_MDM_CORP'],
      dtype='object', length=165)

In [ ]:
test_cdna['v43'] = test_cdna['v43'].fillna('OTHERS')

In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v43'], drop_first=True)

In [ ]:
test_cdna['v54'] = test_cdna['v54'].fillna('Male')

In [ ]:
test_cdna['v54'] = test_cdna['v54'].replace(['MALE', 'Male', 'M'], 'Male')
test_cdna['v54'] = test_cdna['v54'].replace(['FEMALE', 'Female', 'F'], 'Female')
test_cdna['v54'] = test_cdna['v54'].replace(['C', 'OTHERS', 'THIRD GENDER','U','O'], 'Others')

In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v54'], drop_first=False)

In [ ]:
test_cdna=test_cdna.drop(columns=['v54_Others'])

In [ ]:
test_cdna['v99'] = test_cdna['v99'].fillna('Nan')

In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v99'], drop_first=True)

In [ ]:
test_cdna['v101'] = test_cdna['v101'].fillna("Tier 1")

In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v101'], drop_first=False)

In [ ]:
test_cdna=test_cdna.drop(columns=['v101_Tier 8'])

In [ ]:
test_cdna['v278'] = test_cdna['v278'].fillna("Other")

In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v278'], drop_first=False)

In [ ]:
test_cdna=test_cdna.drop(columns=['v278_Other'])

In [ ]:
test_cdna['v286'] = test_cdna['v286'].fillna("Other")

In [ ]:
test_cdna = pd.get_dummies(test_cdna, columns=['v286'], drop_first=False)

In [ ]:
bool_columns = test_cdna.select_dtypes(include='bool').columns
bool_columns

Index(['v39', 'v40', 'v41', 'v45', 'v49', 'v50', 'v51', 'v52', 'v53',
       'v36_FINNONE', 'v36_MDM', 'v36_MDM_CORP', 'v43_OTHERS', 'v43_SINGLE',
       'v43_UNMARRIED', 'v54_Female', 'v54_Male', 'v99_and', 'v99_ios',
       'v101_Rural', 'v101_Tier 1', 'v101_Tier 2', 'v101_Tier 3',
       'v101_Tier 4', 'v101_Tier 5', 'v101_Tier 6', 'v101_Tier 7',
       'v278_EVENING', 'v278_MORNING', 'v286_FRIDAY', 'v286_Other',
       'v286_WEEKDAY', 'v286_WEEKEND'],
      dtype='object')

In [ ]:
test_cdna[bool_columns] =  test_cdna[bool_columns].astype(int)

In [ ]:
null_count=pd.DataFrame(test_cdna.isnull().sum())
null_count.columns=['count']
for index, row in null_count.iterrows():
    for col_name in null_count.columns:
        if (row[col_name]!=0):
            print(index,row[col_name])

v6 75
v11 1
v27 4022
v29 11701
v33 13541
v34 12693
v35 16899
v42 34142
v55 41458
v56 2011
v60 24705
v66 40871
v68 40871
v69 40871
v71 40871
v73 10082
v74 40871
v81 8
v102 11909
v103 41109
v271 22289
v272 22289
v273 22289
v274 22289
v275 22289
v276 22289
v277 22289
v279 22289
v280 22289
v281 22289
v282 22289
v283 22289
v284 22289
v285 22289


In [ ]:
X=X.drop(columns=['v63'])
X_test_data=test_cdna[X.columns]

In [ ]:
null_count=pd.DataFrame(X.isnull().sum())
null_count.columns=['count']
for index, row in null_count.iterrows():
    for col_name in null_count.columns:
        if (row[col_name]!=0):
            print(index,row[col_name])

In [ ]:
test_cdna['CUSTOMER_CODE'].nunique()

68450

# MODEL PREDICTION ON TEST DATA

In [ ]:
import pandas as pd
import numpy as np

predicted_probs = xgb_model.predict_proba(X_test_data)  # Shape: (n_samples, n_classes)

# Decode the class labels
decoded_labels = label_encoder.inverse_transform(range(predicted_probs.shape[1]))  # Decode class indices to original labels

# Create a DataFrame to hold the probabilities with decoded class labels as columns
probs_df = pd.DataFrame(predicted_probs, columns=decoded_labels)

# Add a column for the original row index (optional, for tracking)
probs_df['row_index'] = np.arange(len(predicted_probs))

# Sort probabilities for each row and store only the sorted slot names in a new column
probs_df['sorted_slots'] = probs_df.iloc[:, :-1].apply(
    lambda row: [label for label, _ in sorted(
        zip(decoded_labels, row),  # Pair decoded labels with probabilities
        key=lambda x: x[1],       # Sort by probability
        reverse=True              # Descending order
    )],
    axis=1
)

# Display the sorted slot names
print(probs_df[['row_index', 'sorted_slots']])

In [ ]:
predicted_probs=xgb_model.predict_proba(X_test_data)


In [ ]:
predicted_probs

array([[0.02582023, 0.0303061 , 0.04168471, ..., 0.02688157, 0.01872486,
        0.01967617],
       [0.02160028, 0.02569463, 0.02624833, ..., 0.00786475, 0.01903203,
        0.00639428],
       [0.02004275, 0.02204629, 0.03278297, ..., 0.01393569, 0.02410397,
        0.00757504],
       ...,
       [0.02557364, 0.02504365, 0.03894158, ..., 0.01075028, 0.02321499,
        0.01975087],
       [0.024902  , 0.02735749, 0.016374  , ..., 0.01648688, 0.02816274,
        0.00936789],
       [0.01011964, 0.0221534 , 0.02787501, ..., 0.02029005, 0.02053578,
        0.00761185]], dtype=float32)

In [ ]:
# Decode class indices to original labels
decoded_labels = encoder.inverse_transform(range(predicted_probs.shape[1]))  # Decoded labels like 'slot_1', 'slot_2'

# Create a DataFrame with predicted probabilities
probs_df = pd.DataFrame(predicted_probs, columns=decoded_labels)

# Add row index as a separate column (optional)
probs_df['row_index'] = np.arange(len(predicted_probs))

# Sort probabilities for each row and store sorted slot names with 'slot_' prefix
probs_df['sorted_slots'] = probs_df[decoded_labels].apply(
    lambda row: [f"slot_{label}" for label, _ in sorted(
        zip(range(1, len(decoded_labels) + 1), row),  # Pair slot numbers (1-based index) with probabilities
        key=lambda x: x[1],                           # Sort by probability
        reverse=True                                  # Descending order
    )],
    axis=1
)

# Display the sorted slot names
print(probs_df[['row_index', 'sorted_slots']])



       row_index                                       sorted_slots
0              0  [slot_7, slot_23, slot_6, slot_22, slot_20, sl...
1              1  [slot_7, slot_22, slot_14, slot_19, slot_6, sl...
2              2  [slot_7, slot_20, slot_5, slot_22, slot_17, sl...
3              3  [slot_6, slot_7, slot_14, slot_19, slot_22, sl...
4              4  [slot_6, slot_17, slot_7, slot_22, slot_20, sl...
...          ...                                                ...
68445      68445  [slot_6, slot_7, slot_23, slot_16, slot_20, sl...
68446      68446  [slot_23, slot_6, slot_20, slot_17, slot_7, sl...
68447      68447  [slot_23, slot_7, slot_14, slot_6, slot_20, sl...
68448      68448  [slot_7, slot_16, slot_14, slot_5, slot_20, sl...
68449      68449  [slot_7, slot_22, slot_19, slot_14, slot_5, sl...

[68450 rows x 2 columns]


### Creating Final CSV for submission

In [ ]:
test_cdna['predicted_slots_order']=probs_df['sorted_slots']

In [ ]:
test_cdna.head()

,CUSTOMER_CODE,v5,v6,v11,v24,v27,v29,v32,v33,v34,...,v101_Tier 5,v101_Tier 6,v101_Tier 7,v278_EVENING,v278_MORNING,v286_FRIDAY,v286_Other,v286_WEEKDAY,v286_WEEKEND,predicted_slots_order
0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...,99.0,THANE,2018-04-26T00:00:00.000Z,0.0,Married,Self Employed Business,60.0,294cd6e2886c568796d89c66ccdaea76dbf52e44d7598d...,MUMBAI,...,0,0,0,0,1,0,0,0,1,"[slot_7, slot_23, slot_6, slot_22, slot_20, sl..."
1,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...,99.0,KOLKATA,2017-11-20T00:00:00.000Z,0.0,MARRIED,SALARIED,39.0,678edf6b584a6a19c6cc9c684b56579c4348ae8468c9aa...,KOLKATA,...,1,0,0,0,0,0,1,0,0,"[slot_7, slot_22, slot_14, slot_19, slot_6, sl..."
2,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...,99.0,KOLAR,2022-10-03T00:00:00.000Z,0.0,Unmarried,Salaried,84.0,902fb49da19d011962ab0fef97a954af7e6515bfe940aa...,CHINTAMANI,...,0,0,0,0,1,0,0,0,1,"[slot_7, slot_20, slot_5, slot_22, slot_17, sl..."
3,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...,99.0,BUNDI,2022-08-25T12:02:11.000Z,0.0,Married,Self Employed Business,50.0,38054c0d3abba40082cf35b11b4089b5e1581b756e3750...,BUNDI,...,1,0,0,0,0,0,1,0,0,"[slot_6, slot_7, slot_14, slot_19, slot_22, sl..."
4,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...,99.0,HAPUR,2022-09-23T00:00:00.000Z,0.0,MARRIED,NaN,39.0,f425e5991d132be3f040c9b7d10fbd7a9501e5c3fa1dd9...,HAPUR,...,1,0,0,0,1,0,0,0,1,"[slot_6, slot_17, slot_7, slot_22, slot_20, sl..."


In [ ]:
df_to_submit=test_cdna[['CUSTOMER_CODE','predicted_slots_order']]
df_to_submit.head()

,CUSTOMER_CODE,predicted_slots_order
0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...,"[slot_7, slot_23, slot_6, slot_22, slot_20, sl..."
1,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...,"[slot_7, slot_22, slot_14, slot_19, slot_6, sl..."
2,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...,"[slot_7, slot_20, slot_5, slot_22, slot_17, sl..."
3,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...,"[slot_6, slot_7, slot_14, slot_19, slot_22, sl..."
4,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...,"[slot_6, slot_17, slot_7, slot_22, slot_20, sl..."


In [ ]:
df_to_submit.rename(columns={'CUSTOMER_CODE': 'customer_code'}, inplace=True)
df_to_submit

,customer_code,predicted_slots_order
0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...,"[slot_7, slot_23, slot_6, slot_22, slot_20, sl..."
1,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...,"[slot_7, slot_22, slot_14, slot_19, slot_6, sl..."
2,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...,"[slot_7, slot_20, slot_5, slot_22, slot_17, sl..."
3,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...,"[slot_6, slot_7, slot_14, slot_19, slot_22, sl..."
4,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...,"[slot_6, slot_17, slot_7, slot_22, slot_20, sl..."
...,...,...
68445,4cde59d5b42a2d221037cb2fca0de4b017b92486881c72...,"[slot_6, slot_7, slot_23, slot_16, slot_20, sl..."
68446,537705bf6fab6b290248264b5b34cc3f59352740d98964...,"[slot_23, slot_6, slot_20, slot_17, slot_7, sl..."
68447,48d22d3e52065cb6d54e1b88ec4d5a41b99f19be0427a2...,"[slot_23, slot_7, slot_14, slot_6, slot_20, sl..."
68448,5d8e50df07ecaeb2da40b588aab824e9d33a705e005a2b...,"[slot_7, slot_16, slot_14, slot_5, slot_20, sl..."


### Loading Customer Codes from the test_customers

In [ ]:
df_customers=pd.read_csv("test_customers.csv")

In [ ]:
df_submission_final = pd.merge(
    df_to_submit,
    df_customers,
    left_on='customer_code',
    right_on='CUSTOMER_CODE',
    how='inner'  # Keeps only rows that are present in both DataFrames
)
df_submission_final.drop(columns=['CUSTOMER_CODE'], inplace=True)
df_submission_final.shape

(65182, 2)

In [ ]:
df_submission_final.to_csv("convolve_round2_submission.csv",index=False)

In [ ]:
df_submission_final

,customer_code,predicted_slots_order
0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...,"[slot_7, slot_23, slot_6, slot_22, slot_20, sl..."
1,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...,"[slot_7, slot_22, slot_14, slot_19, slot_6, sl..."
2,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...,"[slot_7, slot_20, slot_5, slot_22, slot_17, sl..."
3,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...,"[slot_6, slot_7, slot_14, slot_19, slot_22, sl..."
4,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...,"[slot_6, slot_17, slot_7, slot_22, slot_20, sl..."
...,...,...
65177,6d859ea95f4d7da20ac2aa66e86d0ace5eeb8171b5cbfc...,"[slot_7, slot_22, slot_14, slot_19, slot_6, sl..."
65178,4cde59d5b42a2d221037cb2fca0de4b017b92486881c72...,"[slot_6, slot_7, slot_23, slot_16, slot_20, sl..."
65179,537705bf6fab6b290248264b5b34cc3f59352740d98964...,"[slot_23, slot_6, slot_20, slot_17, slot_7, sl..."
65180,48d22d3e52065cb6d54e1b88ec4d5a41b99f19be0427a2...,"[slot_23, slot_7, slot_14, slot_6, slot_20, sl..."
